In [ ]:
# 看具体某一天的spread数据
from backtesting import *


In [ ]:
import plotly.graph_objects as go

def plot_bid_price_spread_plotly(cf_depth,
                                 market2_price_col,
                                 market1_price_col='market1_bid_price0',
                                 q_10_bt=None,
                                 q_90_bt=None):
    """
    使用 Plotly 绘制两个市场 bid price 的相对差异（bps），
    并在右侧第二坐标轴上叠加 market1 和 market2 的价格曲线。
    """
    # 计算 spread（bps）
    spread = ((cf_depth[market2_price_col] / cf_depth[market1_price_col]) - 1) * 10000
    # spread = ((cf_depth[market2_price_col] - cf_depth[market1_price_col]))/0.0001
    spread.index = cf_depth.index

    # 去除极端值
    # lower = spread.quantile(0.001)
    # upper = spread.quantile(0.999)
    # spread_clipped = spread.clip(lower=lower, upper=upper)


    spread_clipped = spread.copy()

    # 分位数
    q10 = spread_clipped.quantile(0.10)
    q90 = spread_clipped.quantile(0.90)

    # 将UTC时间转换为北京时间（UTC+8）
    beijing_time = cf_depth.index.tz_localize('UTC').tz_convert('Asia/Shanghai')
    cf_depth['beijing_time'] = beijing_time
    # 创建图
    fig = go.Figure()

    # —— Spread 曲线（左侧 y 轴）——
    fig.add_trace(go.Scatter(
        x=beijing_time,
        y=spread_clipped,
        mode='lines',
        name='Spread (bps)',
        line=dict(color='purple'),
        yaxis='y1'
    ))
    #10% / 90% 分位线
    for val, name, dash_color in [
        (q10, f'10% Quantile: {q10:.2f}', 'red'),
        (q90, f'90% Quantile: {q90:.2f}', 'green'),
        (q_10_bt, f'10% Quantile (backtest): {q_10_bt:.2f}', 'blue'),
        (q_90_bt, f'90% Quantile (backtest): {q_90_bt:.2f}', 'orange'),
    ]:
        if val is not None:
            fig.add_trace(go.Scatter(
                x=[beijing_time[0], beijing_time[-1]],
                y=[val, val],
                mode='lines',
                name=name,
                line=dict(color=dash_color, dash='dash'),
                yaxis='y1'
            ))

    # —— 价格曲线（右侧 y 轴）——
    fig.add_trace(go.Scatter(
        x=beijing_time,
        y=cf_depth[market1_price_col],
        mode='lines',
        name=f'{market1_price_col}(binance)',
        yaxis='y2'
    ))
    fig.add_trace(go.Scatter(
        x=beijing_time,
        y=cf_depth[market2_price_col],
        mode='lines',
        name=f'{market2_price_col}(okx)',
        yaxis='y2'
    ))

    # 布局：定义两个纵轴
    fig.update_layout(
        title=f'Spread & Prices: {market2_price_col} vs {market1_price_col}',
        xaxis=dict(title='Time (Beijing)'),
        yaxis=dict(
            title='Spread (bps)',
            side='left',
            showgrid=False,
        ),
        yaxis2=dict(
            title='Price',
            overlaying='y',
            side='right',
            showgrid=False,
        ),
        hovermode='x unified',
        height=1200,
        width=2400,
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()
    return cf_depth

def plot_bid_price_spread_matplotlib(cf_depth,
                                     market2_price_col,
                                     market1_price_col='market1_bid_price0',
                                     q_10_bt=None,
                                     q_90_bt=None):
    """
    使用 Matplotlib 绘制两个市场 bid price 的相对差异（bps），
    并在右侧第二坐标轴上叠加 market1 和 market2 的价格曲线。
    """
    import matplotlib.pyplot as plt

    # 计算 spread（bps）
    spread = ((cf_depth[market2_price_col] / cf_depth[market1_price_col]) - 1) * 10000
    spread.index = cf_depth.index

    # 去除极端值
    # lower = spread.quantile(0.001)
    # upper = spread.quantile(0.999)
    # spread_clipped = spread.clip(lower=lower, upper=upper)
    spread_clipped = spread.copy()

    # 分位数
    q10 = spread_clipped.quantile(0.10)
    q90 = spread_clipped.quantile(0.90)

    # 将UTC时间转换为北京时间（UTC+8）
    beijing_time = cf_depth.index.tz_localize('UTC').tz_convert('Asia/Shanghai')
    cf_depth['beijing_time'] = beijing_time

    fig, ax1 = plt.subplots(figsize=(24, 12))

    # —— Spread 曲线（左侧 y 轴）——
    ax1.plot(beijing_time, spread_clipped, color='purple', label='Spread (bps)')
    # 10% / 90% 分位线
    for val, name, color in [
        (q10, f'10% Quantile: {q10:.2f}', 'red'),
        (q90, f'90% Quantile: {q90:.2f}', 'green'),
        (q_10_bt, f'10% Quantile (backtest): {q_10_bt:.2f}', 'blue'),
        (q_90_bt, f'90% Quantile (backtest): {q_90_bt:.2f}', 'orange'),
    ]:
        if val is not None:
            ax1.axhline(y=val, color=color, linestyle='--', label=name)

    ax1.set_ylabel('Spread (bps)')
    ax1.set_xlabel('Time (Beijing)')
    ax1.grid(False)

    # —— 价格曲线（右侧 y 轴）——
    ax2 = ax1.twinx()
    ax2.plot(beijing_time, cf_depth[market1_price_col], color='black', label=f'{market1_price_col}(binance)')
    ax2.plot(beijing_time, cf_depth[market2_price_col], color='gray', label=f'{market2_price_col}(okx)')
    ax2.set_ylabel('Price')
    ax2.grid(False)

    # 合并图例
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

    plt.title(f'Spread & Prices: {market2_price_col} vs {market1_price_col}')
    plt.tight_layout()
    plt.show()
    return cf_depth

In [ ]:
def read_cf_depth(ccy, start_date, end_date, exchange1, market1, exchange2, market2, data_source):
    if data_source == 'inner_win':
        market1_depth_path = f'/Users/rayxu/Desktop/Obentech/dcdlData/{exchange1}/books/{ccy}/{market1}'
        market2_depth_path = f'/Users/rayxu/Desktop/Obentech/dcdlData/{exchange2}/books/{ccy}/{market2}'
    elif data_source == 'outer_ssd':
        market1_depth_path = f'/Volumes/T7/Obentech/dcdlData/{exchange1}/books/{ccy}/{market1}'
        market2_depth_path = f'/Volumes/T7/Obentech/dcdlData/{exchange2}/books/{ccy}/{market2}'
    elif data_source =='nuts_mm':
        market1_depth_path = f'/Users/rayxu/Downloads/nuts_am/data/{exchange1}/perp/books/{ccy}'
        market2_depth_path = f'/Users/rayxu/Downloads/nuts_am/data/{exchange2}/perp/books/{ccy}'

    if data_source == 'nuts_mm':
        market1_depth = pd.concat([pd.read_parquet(f'{market1_depth_path}/{ccy}usdt_{dd}_depth5.parquet')
                            for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])        
        market2_depth = pd.concat([pd.read_parquet(f'{market2_depth_path}/{ccy}usdt_{dd}_depth5.parquet')
                            for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])
    else:
        market1_depth = pd.concat([pd.read_csv(f'{market1_depth_path}/{ccy}usdt_{dd}_depth5.csv')
                          for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])        
        market2_depth = pd.concat([pd.read_csv(f'{market2_depth_path}/{ccy}usdt_{dd}_depth5.csv')
                          for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])
    # 不知道为啥24年的数据，是23年的年份，所以先加1年处理
    # market1_depth["received_time"] = pd.to_datetime(market1_depth["received_time"]).apply(lambda x:x+relativedelta(years=1))
    time_col = get_time_col(exchange1,exchange2)
    market1_depth[time_col] = pd.to_datetime(market1_depth[time_col], unit = 'ms')
    market1_depth.set_index(time_col, inplace=True)
    market1_depth['ws_type'] = "market1_depth"
    market1_depth.drop_duplicates(inplace=True)


    # 不知道为啥24年的数据，是23年的年份，所以先加1年处理
    # market2_depth["received_time"] = pd.to_datetime(market2_depth["received_time"]).apply(lambda x:x+relativedelta(years=1))
    market2_depth[time_col] = pd.to_datetime(market2_depth[time_col], unit = 'ms')
    market2_depth.set_index(time_col, inplace=True)
    market2_depth['ws_type'] = "market2_depth"
    market2_depth.drop_duplicates(inplace=True)
    
    market1_depth.columns = rename_columns(list(market1_depth.columns), 'market1_')
    market2_depth.columns = rename_columns(list(market2_depth.columns), 'market2_')
########################################################################################
# 04-21修改： 改用merge_asof
    # 保证时间索引已经在 column 中（因为 merge_asof 不能用 index 作为 on）
    market1_depth = market1_depth.reset_index()
    market2_depth = market2_depth.reset_index()

    # 使用 merge_asof 精准时间对齐，100ms 容差，向后对齐
    cf_depth = pd.merge_asof(
        market1_depth.sort_values(time_col),
        market2_depth.sort_values(time_col),
        on=time_col,
        direction='backward',
        tolerance=pd.Timedelta('100ms'),
        suffixes=('_market1', '_market2')
    )

    cf_depth[['market1_ws_type', 'market2_ws_type']] = cf_depth[['market1_ws_type', 'market2_ws_type']].fillna('')
    cf_depth['ws_type'] = cf_depth['market1_ws_type'] + cf_depth['market2_ws_type']
    
    cf_depth.dropna(inplace=True)
    cf_depth = cf_depth.fillna(method='ffill').assign(
        sp_open=lambda df: df['market2_bid_price1']-df['market1_ask_price1'],
        sp_close=lambda df: df['market2_ask_price1']-df['market1_bid_price1'],
        sr_open=lambda df: df['sp_open']/df['market1_ask_price1'],
        sr_close=lambda df: df['sp_close']/df['market1_bid_price1'],
        # 用midprice计算
        midprice=lambda df: (df['market1_bid_price1'] + df['market1_ask_price1']) / 2
    )
    
    # 计算过去N个tick的收益率（用midprice）
    for n in [10, 50, 100, 300, 600]:
        cf_depth[f'ret_mid_{n/10}s'] = cf_depth['midprice'].pct_change(periods=n)
        cf_depth[f'logret_mid_{n/10}s'] = np.log(cf_depth['midprice'] / cf_depth['midprice'].shift(n))
    
    cf_depth.reset_index(inplace=True)
    cf_depth['received_time_diff_1jump_later'] = cf_depth[time_col].shift(-1) - cf_depth[time_col]
    cf_depth['received_time_diff_1jump_later'] = cf_depth['received_time_diff_1jump_later'].apply(lambda x:x.total_seconds())
    cf_depth.set_index(time_col, inplace=True)
    cf_depth_st_index = cf_depth.index[0]
    cf_depth_et_index = cf_depth.index[-1]
    
    return cf_depth

In [ ]:
def read_cf_depth(ccy, start_date, end_date, exchange1, market1, exchange2, market2, data_source):
    if data_source == 'inner_win':
        market1_depth_path = f'/Users/rayxu/Desktop/Obentech/dcdlData/{exchange1}/books/{ccy}/{market1}'
        market2_depth_path = f'/Users/rayxu/Desktop/Obentech/dcdlData/{exchange2}/books/{ccy}/{market2}'
    elif data_source == 'outer_ssd':
        market1_depth_path = f'/Volumes/T7/Obentech/dcdlData/{exchange1}/books/{ccy}/{market1}'
        market2_depth_path = f'/Volumes/T7/Obentech/dcdlData/{exchange2}/books/{ccy}/{market2}'
    elif data_source =='nuts_mm':
        market1_depth_path = f'/Volumes/T7/data/{exchange1}/perp/books/{ccy}'
        market2_depth_path = f'/Volumes/T7/data/{exchange2}/perp/books/{ccy}'

    if data_source == 'nuts_mm':
        market1_depth = pd.concat([pd.read_parquet(f'{market1_depth_path}/{ccy}usdt_{dd}_depth5.parquet')
                            for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])        
        market2_depth = pd.concat([pd.read_parquet(f'{market2_depth_path}/{ccy}usdt_{dd}_depth5.parquet')
                            for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])
    else:
        market1_depth = pd.concat([pd.read_csv(f'{market1_depth_path}/{ccy}usdt_{dd}_depth5.csv')
                          for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])        
        market2_depth = pd.concat([pd.read_csv(f'{market2_depth_path}/{ccy}usdt_{dd}_depth5.csv')
                          for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])
    # 不知道为啥24年的数据，是23年的年份，所以先加1年处理
    # market1_depth["received_time"] = pd.to_datetime(market1_depth["received_time"]).apply(lambda x:x+relativedelta(years=1))
    time_col = get_time_col(exchange1,exchange2)
    market1_depth[time_col] = pd.to_datetime(market1_depth[time_col], unit = 'ms')
    market1_depth.set_index(time_col, inplace=True)
    market1_depth['ws_type'] = "market1_depth"
    # market1_depth.drop_duplicates(inplace=True)

    # 不知道为啥24年的数据，是23年的年份，所以先加1年处理
    # market2_depth["received_time"] = pd.to_datetime(market2_depth["received_time"]).apply(lambda x:x+relativedelta(years=1))
    market2_depth[time_col] = pd.to_datetime(market2_depth[time_col], unit = 'ms')
    market2_depth.set_index(time_col, inplace=True)
    market2_depth['ws_type'] = "market2_depth"
    # market2_depth.drop_duplicates(inplace=True)
    
    market1_depth.columns = rename_columns(list(market1_depth.columns), 'market1_')
    market2_depth.columns = rename_columns(list(market2_depth.columns), 'market2_')
########################################################################################
# 04-21修改： 改用merge_asof
    # 保证时间索引已经在 column 中（因为 merge_asof 不能用 index 作为 on）
    market1_depth = market1_depth.reset_index()
    market2_depth = market2_depth.reset_index()

    # 使用 merge_asof 精准时间对齐，100ms 容差，向后对齐
    cf_depth = pd.merge_asof(
        market1_depth.sort_values(time_col),
        market2_depth.sort_values(time_col),
        on=time_col,
        direction='backward',
        tolerance=pd.Timedelta('100ms'),
        suffixes=('_market1', '_market2')
    )

    cf_depth[['market1_ws_type', 'market2_ws_type']] = cf_depth[['market1_ws_type', 'market2_ws_type']].fillna('')
    cf_depth['ws_type'] = cf_depth['market1_ws_type'] + cf_depth['market2_ws_type']
    
    cf_depth.dropna(inplace=True)
    cf_depth = cf_depth.fillna(method='ffill').assign(
        sp_open=lambda df: df['market2_bid_price1']-df['market1_ask_price1'],
        sp_close=lambda df: df['market2_ask_price1']-df['market1_bid_price1'],
        sr_open=lambda df: df['sp_open']/df['market1_ask_price1'],
        sr_close=lambda df: df['sp_close']/df['market1_bid_price1'],
        # 用midprice计算
        midprice=lambda df: (df['market1_bid_price1'] + df['market1_ask_price1']) / 2
    )
    
    # 计算过去N个tick的收益率（用midprice）
    for n in [10, 50, 100, 300, 600]:
        cf_depth[f'ret_mid_{n/10}s'] = cf_depth['midprice'].pct_change(periods=n)
        cf_depth[f'logret_mid_{n/10}s'] = np.log(cf_depth['midprice'] / cf_depth['midprice'].shift(n))
    
    # 计算过去60s和5s的波动率，并计算它们的比值
    # 假设tick大约0.1s一个，60s窗口大约600个tick，5s窗口大约50个tick
    # 这里用midprice的对数收益率的rolling std来衡量波动率
    cf_depth['logret_mid_1tick'] = np.log(cf_depth['midprice'] / cf_depth['midprice'].shift(1))
    cf_depth['vol_60s'] = cf_depth['logret_mid_1tick'].rolling(window=600, min_periods=600).std()
    cf_depth['vol_5s'] = cf_depth['logret_mid_1tick'].rolling(window=50, min_periods=50).std()
    cf_depth['vol_60s_div_5s'] = cf_depth['vol_60s'] / cf_depth['vol_5s']
    # 你可以直接用 cf_depth['vol_60s_div_5s'] 这一列

    cf_depth.reset_index(inplace=True)
    cf_depth['received_time_diff_1jump_later'] = cf_depth[time_col].shift(-1) - cf_depth[time_col]
    cf_depth['received_time_diff_1jump_later'] = cf_depth['received_time_diff_1jump_later'].apply(lambda x:x.total_seconds())
    cf_depth.set_index(time_col, inplace=True)
    cf_depth_st_index = cf_depth.index[0]
    cf_depth_et_index = cf_depth.index[-1]
    
    return cf_depth

In [ ]:
def read_cf_depth_using_local_time(ccy, start_date, end_date, exchange1, market1, exchange2, market2, data_source):
    if data_source == 'inner_win':
        market1_depth_path = f'/Users/rayxu/Desktop/Obentech/dcdlData/{exchange1}/books/{ccy}/{market1}'
        market2_depth_path = f'/Users/rayxu/Desktop/Obentech/dcdlData/{exchange2}/books/{ccy}/{market2}'
    elif data_source == 'outer_ssd':
        market1_depth_path = f'/Volumes/T7/Obentech/dcdlData/{exchange1}/books/{ccy}/{market1}'
        market2_depth_path = f'/Volumes/T7/Obentech/dcdlData/{exchange2}/books/{ccy}/{market2}'
    elif data_source == 'nuts_mm':
        market1_depth_path = f'/Volumes/T7/data/{exchange1}/perp/books/{ccy}'
        market2_depth_path = f'/Volumes/T7/data/{exchange2}/perp/books/{ccy}'
    # if data_source == 'nuts_am_on_mac':
    #     market1_depth_path = f'/Users/rayxu/Downloads/nuts_am/data/{exchange1}/perp/books/{ccy}'
    #     market2_depth_path = f'/Users/rayxu/Downloads/nuts_am/data/{exchange2}/perp/books/{ccy}'
        market1_depth = pd.concat([pd.read_parquet(f'{market1_depth_path}/{ccy}usdt_{dd}_depth5.parquet')
                            for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])        
        market2_depth = pd.concat([pd.read_parquet(f'{market2_depth_path}/{ccy}usdt_{dd}_depth5.parquet')
                            for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])
    # else:
    #     market1_depth = pd.concat([pd.read_csv(f'{market1_depth_path}/{ccy}usdt_{dd}_depth5.csv')
    #                       for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])        
    #     market2_depth = pd.concat([pd.read_csv(f'{market2_depth_path}/{ccy}usdt_{dd}_depth5.csv')
    #                       for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])

    time_col = 'received_time'
    market1_depth[time_col] = pd.to_datetime(market1_depth[time_col])
    market1_depth.set_index(time_col, inplace=True)
    market1_depth['ws_type'] = "market1_depth"
    # market1_depth.drop_duplicates(inplace=True)


    market2_depth[time_col] = pd.to_datetime(market2_depth[time_col])
    market2_depth.set_index(time_col, inplace=True)
    market2_depth['ws_type'] = "market2_depth"
    # market2_depth.drop_duplicates(inplace=True)
    
    market1_depth.columns = rename_columns(list(market1_depth.columns), 'market1_')
    market2_depth.columns = rename_columns(list(market2_depth.columns), 'market2_')
    ########################################################################################
    # 04-21修改： 改用merge_asof
    # 保证时间索引已经在 column 中（因为 merge_asof 不能用 index 作为 on）
    market1_depth = market1_depth.reset_index()
    market2_depth = market2_depth.reset_index()

    # 使用 merge_asof 精准时间对齐，100ms 容差，向后对齐
    cf_depth = pd.merge_asof(
        market1_depth.sort_values(time_col),
        market2_depth.sort_values(time_col),
        on=time_col,
        direction='backward',
        tolerance=pd.Timedelta('100ms'),
        suffixes=('_market1', '_market2')
    )

    cf_depth[['market1_ws_type', 'market2_ws_type']] = cf_depth[['market1_ws_type', 'market2_ws_type']].fillna('')
    cf_depth['ws_type'] = cf_depth['market1_ws_type'] + cf_depth['market2_ws_type']
    
    cf_depth.dropna(inplace=True)
    cf_depth = cf_depth.fillna(method='ffill').assign(
        sp_open=lambda df: df['market2_bid_price1']-df['market1_ask_price1'],
        sp_close=lambda df: df['market2_ask_price1']-df['market1_bid_price1'],
        sr_open=lambda df: df['sp_open']/df['market1_ask_price1'],
        sr_close=lambda df: df['sp_close']/df['market1_bid_price1'],
        # 用midprice计算
        midprice=lambda df: (df['market1_bid_price1'] + df['market1_ask_price1']) / 2
    )

    # # 计算过去N个tick的收益率（用midprice）
    # for n in [10, 50, 100, 300, 600]:
    #     cf_depth[f'ret_mid_{n/10}s'] = cf_depth['midprice'].pct_change(periods=n)
    #     cf_depth[f'logret_mid_{n/10}s'] = np.log(cf_depth['midprice'] / cf_depth['midprice'].shift(n))
    
    cf_depth.reset_index(inplace=True)
    # cf_depth['received_time_diff_1jump_later'] = cf_depth[time_col].shift(-1) - cf_depth[time_col]
    # cf_depth['received_time_diff_1jump_later'] = cf_depth['received_time_diff_1jump_later'].apply(lambda x:x.total_seconds())
    cf_depth.set_index(time_col, inplace=True)
    beijing_time = cf_depth.index.tz_localize('UTC').tz_convert('Asia/Shanghai').tz_localize(None)
    cf_depth['beijing_time'] = beijing_time
    cf_depth_st_index = cf_depth.index[0]
    cf_depth_et_index = cf_depth.index[-1]
    
    return cf_depth


In [ ]:
def read_cf_depth_backtesting(ccy, start_date, end_date, exchange1, market1, exchange2, market2, data_source, isUseLocalTime = False):

    if data_source == 'nuts_mm':
        market1_depth_path = f'/Volumes/T7/data/{exchange1}/perp/books/{ccy}'
        market2_depth_path = f'/Volumes/T7/data/{exchange2}/perp/books/{ccy}'
        market1_depth = pd.concat([pd.read_parquet(f'{market1_depth_path}/{ccy}usdt_{dd}_depth5.parquet')
                            for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])        
        market2_depth = pd.concat([pd.read_parquet(f'{market2_depth_path}/{ccy}usdt_{dd}_depth5.parquet')
                            for dd in pd.date_range(start_date, end_date).strftime('%Y-%m-%d')])
    if isUseLocalTime:
        time_col = 'received_time'
        market1_depth[time_col] = pd.to_datetime(market1_depth[time_col])
        market2_depth[time_col] = pd.to_datetime(market2_depth[time_col])
    else:
        time_col = 'T'
        market1_depth[time_col] = pd.to_datetime(market1_depth[time_col], unit='ms')
        market2_depth[time_col] = pd.to_datetime(market2_depth[time_col], unit='ms')

    market1_depth.set_index(time_col, inplace=True)
    market1_depth['ws_type'] = "market1_depth"
    # market1_depth.drop_duplicates(inplace=True)


    market2_depth.set_index(time_col, inplace=True)
    market2_depth['ws_type'] = "market2_depth"
    # market2_depth.drop_duplicates(inplace=True)
    


    ########################################################################################
    # 04-21修改： 改用merge_asof
    # 保证时间索引已经在 column 中（因为 merge_asof 不能用 index 作为 on）
    market1_depth = market1_depth.reset_index()
    market2_depth = market2_depth.reset_index()

    merged_time = pd.Series(
        sorted(set(market1_depth[time_col]) | set(market2_depth[time_col]))
    )


    market1_depth_full = pd.merge_asof(merged_time.to_frame(time_col), market1_depth.sort_values(time_col),
                            on=time_col, direction='backward',tolerance=pd.Timedelta('100ms'))
    market2_depth_full = pd.merge_asof(merged_time.to_frame(time_col), market2_depth.sort_values(time_col),
                            on=time_col, direction='backward',tolerance=pd.Timedelta('100ms'))
                            
    merged_depth = pd.concat(
        [market1_depth_full.add_prefix('market1_'), market2_depth_full.add_prefix('market2_')], axis=1
    )
    merged_depth[time_col] = merged_time.values
    merged_depth['sr_open'] = (merged_depth['market2_ask_price0']/merged_depth['market1_ask_price0']-1)*10000
    merged_depth['sr_close'] = (merged_depth['market2_bid_price0']/merged_depth['market1_bid_price0']-1)*10000
    cf_depth = merged_depth.dropna()

    cf_depth.reset_index(inplace=True)
    cf_depth.set_index(time_col, inplace=True)
    if not isUseLocalTime:
        beijing_time = cf_depth.index.tz_localize('UTC').tz_convert('Asia/Shanghai').tz_localize(None)
        cf_depth['beijing_time'] = beijing_time
    else:
        cf_depth['beijing_time'] = cf_depth.index
        
    cf_depth = cf_depth.sort_index()
    # rolling('5s') 表示时间窗口5秒；center=False表示只看过去
    cf_depth['sr_open_median_5s'] = cf_depth['sr_open'].rolling('5s').median()
    cf_depth['sr_open_median_10s'] = cf_depth['sr_open'].rolling('10s').median()
    cf_depth['sr_open_median_30s'] = cf_depth['sr_open'].rolling('30s').median()
    cf_depth['sr_close_median_5s'] = cf_depth['sr_close'].rolling('5s').median()
    cf_depth['sr_close_median_10s'] = cf_depth['sr_close'].rolling('10s').median()
    cf_depth['sr_close_median_30s'] = cf_depth['sr_close'].rolling('30s').median()
    return cf_depth


In [ ]:
def read_cf_depth_history_1min(ccy, start_date, end_date,data_source):

    ccy = symbol
    # 读聚合后的深度数据（你已有函数）
    cf_depth = pd.read_csv(f'/Volumes/T7/Obentech/historyDepthData/depth_okx_binance_{symbol.lower()}-usdt_1min.csv')
    cf_depth['event_time'] = pd.to_datetime(cf_depth['event_time'], unit='ms')
    cf_depth['beijing_time'] = cf_depth['event_time']+pd.Timedelta(hours=8)
    cf_depth = cf_depth[(cf_depth['beijing_time']>=pd.to_datetime(st)) & (cf_depth['beijing_time']<=pd.to_datetime(et))]
    cf_depth.set_index('beijing_time', inplace=True)
    # 增加最后一个时间点00:00:00（之前是到23:59:59）
    st = pd.to_datetime(st)
    et = pd.to_datetime(et)
    end_midnight = et + pd.Timedelta(days=1) 

    spread_bid = cf_depth['avg_sr_bid']
    spread_ask = cf_depth['avg_sr_ask']
    return cf_depth, spread_bid, spread_ask

In [ ]:
ccy = 'TON'
st='2026-02-11'
et='2026-02-11'
# train_st='2025-07-01'
# train_et='2025-07-18'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "nuts_mm"




cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)
# cf_depth = read_cf_depth_backtesting(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)
# cf_depth = cf_depth[::50]
cf_depth_train = cf_depth.copy()

spread_bid = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
spread_ask = ((cf_depth_train['market2_ask_price0'] / cf_depth_train['market1_ask_price0']) - 1) * 10000
q10_train = spread_bid.quantile(close_quantile)
q90_train = spread_ask.quantile(open_quantile)

target_open_sr = q90_train/10000
target_close_sr = q10_train/10000

# plot_bid_price_spread_plotly(cf_depth, 'market2_ask_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)


In [ ]:
spread_ask.quantile(0.95)

In [ ]:
spread_bid.quantile(0.95)

In [ ]:
spread_ask.quantile(0.95)

In [ ]:
spread_bid.quantile(0.05)

In [ ]:
spread_bid_reverse = (cf_depth_train['market1_bid_price0'] / cf_depth_train['market2_bid_price0'] - 1) * 10000
spread_ask_reverse = (cf_depth_train['market1_ask_price0'] / cf_depth_train['market2_ask_price0'] - 1) * 10000

In [ ]:
print(spread_bid_reverse.quantile(0.05))
print(spread_ask_reverse.quantile(0.95))

In [ ]:
cf_depth['beijing_time'] = cf_depth['beijing_time'].dt.tz_localize(None)
cf_depth['ESRA'] = (cf_depth['market2_ask_price0']/cf_depth['market1_ask_price0']-1)
cf_depth[cf_depth['beijing_time']>pd.to_datetime('2025-09-25 10:10:56')].head(30)

In [ ]:
import numpy as np
import pandas as pd

def count_crossings(spread_series: pd.Series, lower: float, upper: float) -> int:
    in_long = False
    in_short = False
    count = 0

    for x in spread_series:
        if not in_long and x < lower:
            in_long = True
        elif in_long and x > upper:
            count += 1
            in_long = False

        if not in_short and x > upper:
            in_short = True
        elif in_short and x < lower:
            count += 1
            in_short = False

    return count

def find_best_threshold(spread_series: pd.Series, threshold_gap=0.00015, step=5e-5):
    min_spread = -0.0005
    max_spread = 0.0005
    
    best_lower = None
    best_upper = None
    max_count = 0

    for lower in np.arange(min_spread, max_spread - threshold_gap, step):
        upper = lower + threshold_gap
        count = count_crossings(spread_series, lower, upper)
        print(lower,upper,count)
        if count > max_count:
            max_count = count
            best_lower = lower
            best_upper = upper

    return best_lower, best_upper, max_count


In [ ]:
# 假设 spread_series 是你的价差数据
best_lower, best_upper, max_crossings = find_best_threshold(spread_bid/10000, threshold_gap = 0.00015, step = 0.00001)
print(f"Best lower: {best_lower:.5f}, upper: {best_upper:.5f}, crossings: {max_crossings}")


In [ ]:
最近30天：
-0.00008, 0.00007

最近15天：
-0.0001, 0.0005

In [ ]:
q10_train

In [ ]:
q90_train

In [ ]:
# 这行代码的意思是：从 spread_bid 这个 Series 里每隔1000个取一个元素（即每1000个取一个），用于采样查看数据的分布或趋势。
import matplotlib.pyplot as plt

sampled = spread_bid[::100000]
plt.figure(figsize=(20, 10))
plt.plot(sampled.index, sampled.values, linestyle='-')
plt.xlabel('Time')
plt.ylabel('Spread Bid')
plt.title('Spread Bid vs Time ')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
cf_depth = plot_bid_price_spread_matplotlib(cf_depth, 'market2_ask_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)


In [ ]:
q10_train

In [ ]:
q90_train

In [ ]:
cf_depth

In [ ]:
cf_depth.head(300)['market2_ask_price0'].plot()
print(calc_trend_r2(cf_depth.head(300)['market2_ask_price0']))

In [ ]:
cf_depth[100:400]['market2_ask_price0'].plot()
print(calc_trend_r2(cf_depth[100:400]['market2_ask_price0']))

In [ ]:
cf_depth[300:600]['market2_ask_price0'].plot()
print(calc_trend_r2(cf_depth[300:600]['market2_ask_price0']))

In [ ]:
from sklearn.linear_model import LinearRegression
def calc_trend_r2(efficient_rm):
    if efficient_rm.size < 2:       # 少于2个点无法回归
        return 0.0, 0.0
    prices = np.array(list(efficient_rm))[::-1]
    y = (prices / prices[0] - 1)
    X = np.linspace(0, 1, len(prices)).reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    return model.coef_[0], model.score(X, y)

In [ ]:
calc_trend_r2(cf_depth.head(300)['market2_ask_price0'])

In [ ]:
cf_depth = plot_bid_price_spread_plotly(cf_depth, 'market2_ask_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)

In [ ]:
spread_train.quantile(0.8)

In [ ]:
spread_train.quantile(0.2)

In [ ]:
q10_train

In [ ]:
q90_train

In [ ]:
cf_depth

In [ ]:
q10_train

In [ ]:
q90_train

In [ ]:
spread.index

In [ ]:
# 按天统计spread的95%和5%分位，index是datetime格式

# 计算spread
spread = ((cf_depth['market2_bid_price0'] / cf_depth['market1_bid_price0']) - 1) * 10000

# 用index的日期部分直接分组，无需新建date列
# 如果index不是DatetimeIndex，先转一下
if not isinstance(cf_depth.index, pd.DatetimeIndex):
    cf_depth = cf_depth.copy()
    cf_depth.index = pd.to_datetime(cf_depth.index)

# 按天统计95分位和5分位
daily_quantiles = spread.groupby(cf_depth.index.date).quantile([0.05, 0.95]).unstack()
daily_quantiles.columns = ['5%_quantile', '95%_quantile']
daily_quantiles = daily_quantiles.reset_index(names='date')

print(daily_quantiles)

In [ ]:
pd.read_csv('/Users/rayxu/Downloads/cfdc_dcpro1_stat(1).csv').sort_values('net_amount_1')

In [ ]:
q10_train

In [ ]:
plot_bid_price_spread_matplotlib(cf_depth, 'market2_bid_price0','market1_bid_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)

In [ ]:
q10_train

In [ ]:
q90_train


In [ ]:
ccy = 'ALCH'
st='2025-07-15'
et='2025-07-21'
# train_st='2025-07-01'
# train_et='2025-07-03'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "outer_ssd"



# cf_depth_train = read_cf_depth(ccy, train_st, train_et, exchange1, market1, exchange2, market2, data_source=data_source)
cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)


spread_train = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
# spread_train = ((cf_depth_train['market2_bid_price0'] - cf_depth_train['market1_bid_price0']))
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)

target_open_sr = q90_train/10000
target_close_sr = q10_train/10000

# plot_bid_price_spread_plotly(cf_depth, 'market2_bid_price0','market1_bid_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)
plot_bid_price_spread_plotly(cf_depth, 'market2_bid_price0','market1_bid_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)


In [ ]:
df_00 = cf_depth[(cf_depth.index>=pd.to_datetime('2025-07-02 15:00:00'))&(cf_depth.index<pd.to_datetime('2025-07-02 16:00:00'))].copy()
spread_train = ((df_00['market2_bid_price0'] / df_00['market1_bid_price0']) - 1) * 10000
print(spread_train.quantile(0.05),spread_train.quantile(0.95))

In [ ]:
df_01 = cf_depth[(cf_depth.index>=pd.to_datetime('2025-07-01 17:00:00'))&(cf_depth.index<pd.to_datetime('2025-07-02 17:00:00'))].copy()
spread_train = ((df_01['market2_bid_price0'] / df_01['market1_bid_price0']) - 1) * 10000
print(spread_train.quantile(0.05),spread_train.quantile(0.95))

In [ ]:
df_02 = cf_depth[(cf_depth.index>=pd.to_datetime('2025-07-01 18:00:00'))&(cf_depth.index<pd.to_datetime('2025-07-02 18:00:00'))].copy()
spread_train = ((df_02['market2_bid_price0'] / df_02['market1_bid_price0']) - 1) * 10000
print(spread_train.quantile(0.05),spread_train.quantile(0.95))

In [ ]:
spread_train.apply(type).value_counts()

In [ ]:
spread_train.quantile(0.05)

In [ ]:
spread_train.quantile(0.2)

In [ ]:
spread_train.quantile(0.8)

In [ ]:
spread_train.quantile(0.9)

In [ ]:
spread_train.quantile(0.95)

In [ ]:
spread_train.value_counts().head(20)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(spread_train, bins=200)  # 把bins设得很大，比如200
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram with many bins')
plt.show()


In [ ]:
cf_depth

In [ ]:
ccy = 'BTC'
st='2025-07-02'
et='2025-07-15'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "outer_ssd"


cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)

In [ ]:
cf_depth

In [ ]:
show_cols = ['index', 'market1_received_time', 'market1_E', 'market1_bid_price0',
       'market1_bid_size0',
       'market1_ask_price0', 'market1_ask_size0','market1_ws_type', 'market2_received_time',
       'market2_E', 'market2_bid_price0', 'market2_bid_size0',
        'market2_ask_price0',
       'market2_ask_size0',
       'market2_ws_type', 'ws_type', 'sp_open', 'sp_close', 'sr_open',
       'sr_close', 'midprice', 'ret_mid_1.0s', 'logret_mid_1.0s','ret_mid_5.0s',
       'ret_mid_10.0s', 'logret_mid_10.0s', 'ret_mid_30.0s',
       'logret_mid_30.0s', 'ret_mid_60.0s', 'logret_mid_60.0s']

In [ ]:
cf_depth[show_cols].to_csv('/Users/rayxu/Desktop/Obentech/cf_depth_BTC.csv')

In [ ]:
ccy = 'BTC'
st='2025-07-11'
et='2025-07-11'
train_st='2025-07-11'
train_et='2025-07-11'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "outer_ssd"



cf_depth_train = read_cf_depth(ccy, train_st, train_et, exchange1, market1, exchange2, market2, data_source=data_source)
cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)


spread_train = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)

target_open_sr = q90_train/10000
target_close_sr = q10_train/10000

cf_depth = plot_bid_price_spread_plotly(cf_depth, 'market2_ask_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)


In [ ]:
cf_depth

In [ ]:
spread_train = ((cf_depth_train['market1_bid_price0'] / cf_depth_train['market2_bid_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)

In [ ]:
q10_train

In [ ]:
q90_train

In [ ]:
ccy = 'ETH'
st='2025-11-01'
et='2025-11-05'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "nuts_mm"




# cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)
cf_depth = read_cf_depth_backtesting(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)
cf_depth_train = cf_depth.copy()

spread_bid = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
spread_ask = ((cf_depth_train['market2_ask_price0'] / cf_depth_train['market1_ask_price0']) - 1) * 10000
q10_train = spread_bid.quantile(close_quantile)
q90_train = spread_ask.quantile(open_quantile)

target_open_sr = q90_train/10000
target_close_sr = q10_train/10000

# plot_bid_price_spread_plotly(cf_depth, 'market2_ask_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)


In [ ]:
# 帮我只画一下beijingtime 20250710 21:59 - 20250710 22:05的图
start_time = pd.Timestamp('2025-07-11 05:50:00', tz='Asia/Shanghai')
end_time = pd.Timestamp('2025-07-11 05:55:00', tz='Asia/Shanghai')
cf_depth_sel = cf_depth[(cf_depth['beijing_time'] >= start_time) & (cf_depth['beijing_time'] <= end_time)]
plot_bid_price_spread_plotly(cf_depth_sel, 'market2_ask_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)

In [ ]:
ccy = 'ETH'
st='2025-07-09'
et='2025-07-09'
train_st='2025-07-09'
train_et='2025-07-09'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "outer_ssd"



cf_depth_train = read_cf_depth(ccy, train_st, train_et, exchange1, market1, exchange2, market2, data_source=data_source)
cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)


spread_train = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)

target_open_sr = q90_train/10000
target_close_sr = q10_train/10000

plot_bid_price_spread_plotly(cf_depth, 'market1_bid_price0','market2_bid_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)


In [ ]:

spread_train = ((cf_depth_train['market1_bid_price0'] / cf_depth_train['market2_bid_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)
print(q10_train,q90_train)

In [ ]:

spread_train = ((cf_depth_train['market1_ask_price0'] / cf_depth_train['market2_ask_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)
print(q10_train,q90_train)

In [ ]:

spread_train = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)
print(q10_train,q90_train)

In [ ]:
q10_train

In [ ]:
q90_train

In [ ]:
ccy = 'ETH'
st='2025-07-07'
et='2025-07-07'
train_st='2025-07-07'
train_et='2025-07-07'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "outer_ssd"



cf_depth_train = read_cf_depth(ccy, train_st, train_et, exchange1, market1, exchange2, market2, data_source=data_source)
cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)


spread_train = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)

target_open_sr = q90_train/10000
target_close_sr = q10_train/10000

plot_bid_price_spread_plotly(cf_depth, 'market2_bid_price0','market1_bid_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)


In [ ]:
cf_depth = plot_bid_price_spread_plotly(cf_depth, 'market2_bid_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)

In [ ]:
ccy = 'ETH'
st='2025-07-08'
et='2025-07-08'
train_st='2025-07-08'
train_et='2025-07-08'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "outer_ssd"



cf_depth_train = read_cf_depth(ccy, train_st, train_et, exchange1, market1, exchange2, market2, data_source=data_source)
cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)


spread_train = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)

target_open_sr = q90_train/10000
target_close_sr = q10_train/10000

plot_bid_price_spread_plotly(cf_depth, 'market2_bid_price0','market1_bid_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)


In [ ]:
0.031140+2*0.00001

In [ ]:
# 把index为 152320 和 152330的行在dataframe里面标黄出来

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
cf_depth['spread'] = (cf_depth['market2_bid_price0'] / cf_depth['market1_ask_price0']-1)*10000
# cf_depth['spread_real'] = ((cf_depth['market2_ask_price0']/(cf_depth['market1_bid_price0']+2*0.00001))-1)*10000
# 把index为 152320 和 152330的行在dataframe里面标黄出来

# 创建一个样式对象来高亮显示特定行
def highlight_rows(row):
    if row['index'] in [47714, 47725]:
        return ['background-color: yellow'] * len(row)
    return [''] * len(row)

# # 应用样式并显示DataFrame
# # 注意：只有在Jupyter notebook中才能显示带样式的DataFrame
# styled_df = cf_depth[cf_depth.index>pd.Timestamp('2025-06-13 00:07:30')].head(20).style.apply(highlight_rows, axis=1)
# styled_df
temp = cf_depth[cf_depth.beijing_time>pd.Timestamp('2025-07-07 05:41:12', tz='Asia/Shanghai')].head(40)
temp = temp[~temp.index.duplicated(keep = 'last')]
temp.style.apply(highlight_rows, axis=1)



In [ ]:
cf_depth[cf_depth.index>pd.Timestamp('2025-06-13 17:19:50')].head(40)

In [ ]:
df_binance = pd.read_csv('/Volumes/T7/Obentech/dcdlData/binance/books/VINE/swap/vineusdt_2025-06-13_depth5.csv')
df_binance['T'] = pd.to_datetime(df_binance['T'], unit = 'ms')
df_binance['E'] = pd.to_datetime(df_binance['E'], unit = 'ms')
df_binance[df_binance['T']>pd.Timestamp('2025-06-13 00:07:30')].head(20)

In [ ]:
df_okx = pd.read_csv('/Volumes/T7/Obentech/dcdlData/okx/books/VINE/swap/vineusdt_2025-06-13_depth5.csv')
df_okx['T'] = pd.to_datetime(df_okx['T'], unit = 'ms')
# df_okx['E'] = pd.to_datetime(df_okx['E'], unit = 'ms')
df_okx[df_okx['T']>pd.Timestamp('2025-06-13 00:07:30')].head(20)

In [ ]:
st='2025-06-12'
et='2025-06-13'
train_st='2025-06-11'
train_et='2025-06-11'
exchange1='binance'
market1='swap'
exchange2='okx'
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
data_source = 'outer_ssd'
market1_stop_loss_sr_delta = -0.003
market2_stop_loss_sr_delta = -0.003
open_replace_tick_num = 2
close_replace_tick_num = 2
open_tick_num = 1
close_tick_num = 1
stats, cum_pnl, trade_pnl, funding_pnl = run_arbitrage_workflow_MT(
    'VINE', 
    st=st, 
    et=et,
    train_st= train_st,
    train_et= train_et,
    exchange1= exchange1, 
    market1= market1,
    exchange2= exchange2,      
    market2= market2,
    open_quantile = open_quantile,
    close_quantile = close_quantile,
    data_source = data_source,
    open_replace_tick_num = open_replace_tick_num,
    close_replace_tick_num = close_replace_tick_num,
    open_tick_num = open_tick_num,
    close_tick_num = close_tick_num
)

In [ ]:
stats

In [ ]:
def maker_maker_backtest_rolling_0605(cf_depth, mt_params, param_df, pos_limit = np.inf):


    open_replace_tick_num = mt_params["open_replace_tick_num"]
    close_replace_tick_num = mt_params["close_replace_tick_num"]


    market1_stop_loss_sr_delta = mt_params['market1_stop_loss_sr_delta']
    market2_stop_loss_sr_delta = mt_params['market2_stop_loss_sr_delta']

    

    tick_size1 = infer_tick_size_from_column(cf_depth['market1_bid_price0'])
    tick_ratio1 = round(tick_size1/cf_depth.iloc[-1]['market1_bid_price0'], 6)
    round_num1 = int(np.log10(1/tick_size1))
    tick_size2 = infer_tick_size_from_column(cf_depth['market2_bid_price0'])
    tick_ratio2 = round(tick_size2/cf_depth.iloc[-1]['market2_bid_price0'], 6)
    round_num2 = int(np.log10(1/tick_size2))
    print("交易所1精度信息", tick_size1, tick_ratio1, round_num1)
    print("交易所2精度信息", tick_size2, tick_ratio2, round_num2)
    
    is_open_market1_done = False
    is_open_market2_done = False
    is_open_market1_maker = False
    is_open_market2_maker = False
    open_market1_maker_price = None
    open_market1_taker_price = None
    open_market2_maker_price = None
    open_market2_taker_price = None
    market1_open_time = None
    market2_open_time = None

    is_close_market1_done = False
    is_close_market2_done = False
    is_close_market1_maker = False
    is_close_market2_maker = False
    close_market1_maker_price = None
    close_market1_taker_price = None
    close_market2_maker_price = None
    close_market2_taker_price = None
    market1_close_time = None
    market2_close_time = None
    
    sr_open = None
    sr_open_real = None
    sr_close = None
    sr_close_real = None


    # 回测输出变量
    open_trades = []
    close_trades = []
    position_count = 0

    m1_ask_arr = cf_depth['market1_ask_price0'].values
    m1_bid_arr = cf_depth['market1_bid_price0'].values
    m2_ask_arr = cf_depth['market2_ask_price0'].values
    m2_bid_arr = cf_depth['market2_bid_price0'].values
    idx_arr = cf_depth.index.to_numpy()

    for i in tqdm(range(len(cf_depth)), desc='Processing'):
        idx = idx_arr[i]
        market1_ask_price1 = m1_ask_arr[i]
        market1_bid_price1 = m1_bid_arr[i]
        market2_ask_price1 = m2_ask_arr[i]
        market2_bid_price1 = m2_bid_arr[i]
    # # for idx, ds in tqdm(cf_depth.iterrows(), total=len(cf_depth), desc='Processing'):
    # #     #新加仓位限制

    #     market1_ask_price1 = ds['market1_ask_price0']
    #     market1_bid_price1 = ds['market1_bid_price0']
    #     market2_ask_price1 = ds['market2_ask_price0']
    #     market2_bid_price1 = ds['market2_bid_price0']
        # print(f'{idx}, market1 ask:{market1_ask_price1} , market1 bid: {market1_bid_price1}')

        current_time = idx
        # 新加动态threshold的逻辑
        update_time = param_df.index[param_df.index <= current_time].max()
        sr_open_threshold = param_df.loc[update_time, 'adjusted_open']/10000
        sr_close_threshold = param_df.loc[update_time, 'adjusted_close']/10000

        # 改成TM的价差会好一点
        sr_open = (market2_bid_price1)/(market1_ask_price1 - tick_size1 * 1) - 1
        sr_close = (market2_ask_price1)/(market1_bid_price1 + tick_size1 * 1) - 1


        # 把开平仓预设撤单价差作为撤单条件(实盘逻辑)
        sr_open_cancel_threshold = sr_open_threshold
        sr_close_cancel_threshold = sr_close_threshold
        
        # 下单判断
        open_judgement = (sr_open >= sr_open_threshold)
        close_judgement = (sr_close <= sr_close_threshold)
        # 撤单判断
        open_cancel_judgement = (sr_open < sr_open_cancel_threshold)
        close_cancel_judgement = (sr_close > sr_close_cancel_threshold)

        is_any_trade_in_progress = (
            is_open_market1_maker  or 
            is_close_market1_maker 
        )




        if open_judgement and (not is_any_trade_in_progress) and (not is_open_market1_maker) and position_count < pos_limit:
            open_market1_maker_price = round(market1_ask_price1 - tick_size1 * 1, round_num1)  #bid
            is_open_market1_maker = True
            # print(f'{idx},开仓挂单，当前价差: {sr_open}, {sr_close}, 当前threshold: {sr_open_threshold},{sr_close_threshold}, 开仓挂单，market1挂单价:{open_market1_maker_price}')
        # 如果价差消失且没成交，撤单
        if open_cancel_judgement and is_open_market1_maker:
            is_open_market1_maker = False
            # print(f'{idx},价差消失，开仓撤单，当前价差: {sr_open}, 当前threshold: {sr_open_threshold}')

        # 如果双边都没有成交
        if (is_open_market1_maker and (not is_open_market1_done)):
            # 判断market1是否成交
            if (market1_ask_price1 <= open_market1_maker_price):
                is_open_market1_done = True
                is_open_market1_maker = False
                market1_open_time = idx
                # print(f'{idx}, market1Maker成交：{open_market1_maker_price}')
            if (market1_ask_price1 - open_market1_maker_price)> tick_size1 * open_replace_tick_num:         # 这个地方等于的逻辑是不是有点问题？ 如果价格不变的话，好像也在重新挂单。。。
                # print(f'market1_ask_price1: {market1_ask_price1}, open_market1_maker_price: {open_market1_maker_price}, tick_size1: {tick_size1}, open_replace_tick_num: {open_replace_tick_num}')
                open_market1_maker_price = round(market1_ask_price1 - tick_size1 * 1, round_num1)           # 这个地方是不是应该再进行一下撤单判断？ 不然有可能直接成交了； 好像不需要，因为到这一步了就说明market1，2都没有成交，就直接continue了。
                # print(f'{idx}, market1Maker重新挂单，当前行为: buy, 当前ask_price: {market1_ask_price1}, 当前价差: {sr_open},当前threshold: {sr_open_threshold}, market1挂单价:{open_market1_maker_price}')
        # 如果market1成交了
        if is_open_market1_done and (not is_open_market2_done):
            if open_market1_maker_price is None:
                print(f'open_market1_maker_price is None at {idx}')
            if sr_open_threshold is None:
                print(f'sr_open_threshold is None at {idx}')
            # 判读market2是否要taker对冲掉
            if (market2_bid_price1/open_market1_maker_price - 1) <= (sr_open_threshold - market2_stop_loss_sr_delta):
                open_market2_taker_price = market2_bid_price1
                is_open_market2_done = True
                sr_open_real = open_market2_taker_price/open_market1_maker_price - 1
                market2_open_time = idx
                open_trades.append([idx, sr_open, sr_open_real, open_market1_maker_price, open_market2_taker_price,
                                    'market1_maker', 'market2_taker',
                                    market1_open_time, market2_open_time])
                # print(f'{idx}, market2Taker对冲， market2对冲价格：{open_market2_taker_price}')
            # 判断market2是否成交
            # elif market2_bid_price1 >= open_market2_maker_price:
            #     is_open_market2_done = True
            #     sr_open_real = open_market2_maker_price/open_market1_maker_price - 1
            #     market2_open_time = idx
            #     open_trades.append([idx, sr_open, sr_open_real, open_market1_maker_price, open_market2_maker_price,
            #                         'market1_maker', 'market2_maker',
            #                         market1_open_time, market2_open_time])
            #     print('sb')

        # 成交完成，重置
        if is_open_market1_done and is_open_market2_done:
            position_count += 1
            is_open_market1_done = False
            is_open_market2_done = False
            is_open_market1_maker = False
            open_market1_maker_price = None
            open_market2_maker_price = None
            open_market2_taker_price = None
            market1_open_time = None
            market2_open_time = None


        # 平仓
        # print(close_judgement,is_close_market1_maker,is_close_market2_maker)
        if close_judgement and (not is_any_trade_in_progress) and (not is_close_market1_maker) and (position_count > -pos_limit):
            close_market1_maker_price = round(market1_bid_price1 + tick_size1 * 1, round_num1)
            is_close_market1_maker = True
            print(f'{idx}, 平仓挂单，当前价差: {sr_open}, {sr_close}, 当前threshold: {sr_open_threshold},{sr_close_threshold}, market1挂单价:{close_market1_maker_price}')

        # 如果价差消失且没成交，撤单
        if close_cancel_judgement and is_close_market1_maker:
            is_close_market1_maker = False
            # print(f'{idx},价差消失，平仓撤单，当前价差: {sr_close}, 当前threshold: {sr_close_threshold}')

        # 如果边都没有成交
        if (is_close_market1_maker and (not is_close_market1_done)):
            # 判断market1是否成交
            if (market1_bid_price1 >= close_market1_maker_price):
                is_close_market1_done = True
                is_close_market1_maker = False
                market1_close_time = idx
                print(f'{idx}, market1Maker成交：{close_market1_maker_price}')
            # 判断是否重新挂单
            if (close_market1_maker_price - market1_bid_price1)> tick_size1 * close_replace_tick_num:
                # print(f'market1_bid_price1: {market1_bid_price1}, close_market1_maker_price: {close_market1_maker_price}, tick_size1: {tick_size1}, close_replace_tick_num: {close_replace_tick_num}')
                close_market1_maker_price = round(market1_bid_price1 + tick_size1 * close_replace_tick_num, round_num1)
                # print(f'{idx}, market1Maker重新挂单，当前行为: sell, 当前bid_price: {market1_bid_price1}, 当前价差: {sr_close},当前threshold: {sr_close_threshold}, market1新的挂单价:{close_market1_maker_price}')

        # 如果market1成交了
        if is_close_market1_done and (not is_close_market2_done):
            # 判断market2是否要taker对冲掉
            if (market2_ask_price1/close_market1_maker_price - 1) >= (sr_close_threshold + market2_stop_loss_sr_delta):
                close_market2_taker_price = market2_ask_price1
                is_close_market2_done = True
                is_close_market2_maker = False
                sr_close_real = close_market2_taker_price/close_market1_maker_price - 1
                close_trades.append([idx, sr_close, sr_close_real, close_market1_maker_price, close_market2_taker_price,
                                     'market1_maker', 'market2_taker',
                                     market1_close_time, market2_close_time])
                print(f'{idx}, marke2Taker成交：{close_market2_taker_price}')
            # # 判断market2是否成交
        #     elif market2_ask_price1 <= close_market2_maker_price:
        #         is_close_market2_done = True
        #         is_close_market2_maker = False
        #         sr_close_real = close_market2_maker_price/close_market1_maker_price - 1
        #         market2_close_time = idx
        #         close_trades.append([idx, sr_close, sr_close_real, close_market1_maker_price, close_market2_maker_price,
        #                              'market1_maker', 'market2_maker',
        #                              market1_close_time, market2_close_time])
        # ###        print(f'{index}, marke2Maker成交：{close_market2_maker_price}')
        #     # 判断market2是否重挂
        #     elif (market2_ask_price1 - close_market2_maker_price)> tick_size2 * close_replace_tick_num:
        #             close_market2_maker_price = round(market2_ask_price1 - tick_size2 * 1, round_num2)
            
        # 成交完成，重置
        if is_close_market1_done and is_close_market2_done:
            position_count -= 1
            is_close_market1_done = False
            is_close_market2_done = False
            is_close_market1_maker = False
            is_close_market2_maker = False
            close_market1_maker_price = None
            close_market2_maker_price = None
            close_market2_taker_price = None
            market1_close_time = None
            market2_close_time = None
            

        # except Exception as e:
                # traceback.print_exc()
    
    open_trades_df = pd.DataFrame(open_trades, columns=['index', 'sr_open', 'sr_open_real', 
                                                        'market1_traded_price', 'market2_traded_price',
                                                        'market1_traded_type', 'market2_traded_type',
                                                        'market1_open_time', 'market2_open_time'])
    open_trades_df.set_index('index', inplace=True)
    close_trades_df = pd.DataFrame(close_trades, columns=['index', 'sr_close', 'sr_close_real', 
                                                        'market1_traded_price', 'market2_traded_price',
                                                        'market1_traded_type', 'market2_traded_type',
                                                        'market1_close_time', 'market2_close_time'])
    close_trades_df.set_index('index', inplace=True)

    
    
    return open_trades_df, close_trades_df
   

In [ ]:
ccy = 'VINE'
if data_source == 'inner_win':
    funding_okx_csv = f'/Users/rayxu/Desktop/Obentech/fundingRateData/okx/{ccy}-USDT-SWAP.csv'
    funding_binance_csv = f'/Users/rayxu/Desktop/Obentech/fundingRateData/binance/{ccy}USDT.csv'
elif data_source == 'outer_ssd':
    funding_okx_csv = f'/Volumes/T7/Obentech/fundingRateData/okx/{ccy}-USDT-SWAP.csv'
    funding_binance_csv = f'/Volumes/T7/Obentech/fundingRateData/binance/{ccy}USDT.csv'        

df_okx      = process_funding_time(funding_okx_csv,  exchange='okx')
df_binance  = process_funding_time(funding_binance_csv,  exchange='binance')

fr_okx      = funding_df_to_series(df_okx)       # OKX
fr_binance  = funding_df_to_series(df_binance)   # Binance



time_index = "local_time"
cf_depth = read_cf_depth(ccy, train_st, et, exchange1, market1, exchange2, market2, data_source=data_source)


# spread_train = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
# capital = 0
mean_price =0.5*(cf_depth['market2_bid_price0'].dropna().mean() + cf_depth['market1_bid_price0'].dropna().mean())

capital = 10000
single_order_amount = 100
single_order_volume = 100/ mean_price
notional = single_order_volume
pos_limit = capital/(2*single_order_amount)
look_back_days = 3
z_threshold = 1.64


param_df = generate_param_df(cf_depth,df_okx, df_binance,ccy,look_back_days,z_threshold,freq= '1D')

# 让cf_depth的日期大于等于st 加上16小时
cf_depth = cf_depth[cf_depth.index >= pd.to_datetime(st) - pd.Timedelta(hours=8)]
cf_depth = cf_depth[~cf_depth.index.duplicated(keep='last')]

taker_traded_delay_seconds = 0.02
maker_ordered_delay_seconds = 0.01


mt_params = {'open_tick_num':open_tick_num, 'open_replace_tick_num':open_replace_tick_num,
            'close_tick_num':close_tick_num, 'close_replace_tick_num':close_replace_tick_num, 
            'taker_traded_delay_seconds':taker_traded_delay_seconds, 'maker_ordered_delay_seconds':maker_ordered_delay_seconds,
            'market1_stop_loss_sr_delta':market1_stop_loss_sr_delta, 'market2_stop_loss_sr_delta':market2_stop_loss_sr_delta,
            'exchange1_type':exchange1, 'market1_type':market1, 'exchange2_type':exchange2, 'market2_type':market2}

open_trades_df, close_trades_df = maker_maker_backtest_rolling_0605(cf_depth, mt_params,param_df,pos_limit= pos_limit)


In [ ]:
def highlight_rows(row):
    if row.name == pd.to_datetime('2025-06-13 00:07:30.590000'):
        return ['background-color: yellow'] * len(row)
    return [''] * len(row)

close_trades_df.tail(20).style.apply(highlight_rows, axis=1)

In [ ]:
plot_bid_price_spread_plotly(cf_depth, 'market2_ask_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)

In [ ]:
ccy = 'VINE'
st='2025-06-11'
et='2025-06-13'
train_st='2025-06-11'
train_et='2025-06-11'
exchange1='binance' 
market1='swap'
exchange2='okx'     
market2='swap'
open_quantile = 0.95
close_quantile = 0.05
order_type = 'MT'
data_source = "outer_ssd"



cf_depth_train = read_cf_depth(ccy, train_st, train_et, exchange1, market1, exchange2, market2, data_source=data_source)
cf_depth = read_cf_depth(ccy, st, et, exchange1, market1, exchange2, market2, data_source=data_source)


spread_train = ((cf_depth_train['market2_bid_price0'] / cf_depth_train['market1_bid_price0']) - 1) * 10000
q10_train = spread_train.quantile(close_quantile)
q90_train = spread_train.quantile(open_quantile)

target_open_sr = q90_train/10000
target_close_sr = q10_train/10000

plot_bid_price_spread_plotly(cf_depth, 'market2_ask_price0','market1_ask_price0',q_10_bt=target_close_sr*10000,q_90_bt=target_open_sr*10000)

In [ ]:
symbol_pnl_dict (1).pkl

In [ ]:
import pickle

# 打开pickle文件
with open('/Users/rayxu/Downloads/symbol_pnl_dict (1).pkl', 'rb') as f:
    data = pickle.load(f)

data